In [1]:
import numpy as np
from collections import Counter
import json
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from newshackathon.dataloading.data_constructor import construct_data_set 

ModuleNotFoundError: No module named 'newshackathon'

In [2]:
train_data=constructDataSet()

In [62]:
train_data

array([[ 8.,  3.,  2., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 2.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 3.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [146]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sklearn as sklearn
%matplotlib inline

In [147]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split




In [148]:
X = train_data[:,:-1]

In [149]:
Y = train_data[:,-1]

In [150]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state=42)


In [151]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

accuracy_score(Y_test, gnb.predict(X_test))

0.68235294117647061

In [152]:
rf = RandomForestClassifier(max_features=200,max_depth=300)
rf.fit(X_train, Y_train)

accuracy_score(Y_test, rf.predict(X_test))


0.72941176470588232

In [153]:
svc_poly = SVC(kernel="poly").fit(X_train, Y_train)
svc_poly.score(X_test, Y_test)

0.54117647058823526

In [154]:
svc_linear = SVC(kernel="linear").fit(X_train, Y_train)
svc_linear.score(X_test, Y_test)

0.81176470588235294

In [155]:
svc_rbf = SVC(kernel="rbf").fit(X_train, Y_train)
svc_rbf.score(X_test, Y_test)

0.6470588235294118